In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext,PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
from huggingface_hub import notebook_login

ModuleNotFoundError: No module named 'torch'

In [19]:
documents = SimpleDirectoryReader("D:\Curious\RAG-Chatbot-LLama2\data").load_data()
documents

[Document(id_='542d174f-986a-4c7f-82f3-99f633a0952e', embedding=None, metadata={'page_label': '1', 'file_name': 'Stripe_2023_annual_letter.pdf', 'file_path': 'D:\\Curious\\RAG-Chatbot-LLama2\\data\\Stripe_2023_annual_letter.pdf', 'file_type': 'application/pdf', 'file_size': 1012921, 'creation_date': '2024-04-15', 'last_modified_date': '2024-04-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='March 13, 2024\nDear Stripe community:\nCharlie Munger described a two-part rule that works wonders in business, science, and elsewhere:  \n1) take a simple idea and 2) take it very seriously.\nStripe’s mission is to grow the GDP of the internet. The core idea behind the company—one we endeavor \nto take very seriously—is that we’re still early in the journey of

In [20]:
system_prompt ="""
You are an Q&A assistant. Your goal is to answer questions as accurately as possible
based on the instructions and context provided.
"""
#Default format supporated by LLama2
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [21]:
notebook_login()

In [28]:
import torch

llm = HuggingFaceLLM(
    context_window = 4096,
    max_new_tokens=256,
    generate_kwargs={"temperature":0.0, "do_sample":False},
    system_prompt= system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map ='auto',
    # torch_dtype=torch.float16
)

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like meta-llama/Llama-2-7b-chat-hf is not the path to a directory containing a file named model-00001-of-00002.safetensors.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
from llama_index.legacy.embeddings.langchain import LangchainEmbedding

In [ ]:
embed_model = LangchainEmbedding(HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model = embed_model
)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("What is stripe doing")